In [64]:
import torch
import torchvision
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import utils
import numpy as np
import os
from collections import defaultdict
import pickle

%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
transform = torchvision.transforms.Compose(
    [
        torchvision.transforms.ToTensor()
    ]
)

## Resnet

### Edit Model for Last Layer Hook

In [42]:
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT).to(device)

In [43]:
resnet18

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [44]:
resnet18.layer4[1]

BasicBlock(
  (conv1): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (conv2): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn2): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [45]:
last_activations = []
def get_last_layer_activation(module, input, output):
    last_activations.append(output)

In [46]:
resnet18.layer4[1].register_forward_hook(get_last_layer_activation)

### Run Inference

In [ ]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)
similaritiesdict = defaultdict(list)
last_activations = []

In [ ]:
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if not 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['reversal']-------------------
-------------Running ['mirror']-------------------
-------------Running ['randomphasic']-------------------
-------------Running ['randomdelete']-------------------
-------------Running ['reversal', 'mirror']-------------------
-------------Running ['randomdelete', 'mirror']-------------------
-------------Running ['reversal', 'randomphasic']-------------------
-------------Running ['randominsertion', 'randomphasic']-------------------
-------------Running ['randomdelete', 'reversal', 'randomphasic']-------------------
-------------Running ['randomdelete', 'reversal', 'mirror']-------------------
-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


In [79]:
with open(f'similarities/exhaustive/resnet/filled.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [90]:
pkl11path = 'mappings/exhaustive_mapping_11.pkl'
pkl11 = utils.load_dict(pkl11path)
similaritiesdict = defaultdict(list)
last_activations = []

In [91]:
folpath = 'exhaustive_generated_set\\11'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl11[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


In [94]:
with open(f'similarities/exhaustive/resnet/unfilled11.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [101]:
pkl10path = 'mappings/exhaustive_mapping_10.pkl'
pkl10 = utils.load_dict(pkl10path)
similaritiesdict = defaultdict(list)
last_activations = []

In [102]:
folpath = 'exhaustive_generated_set\\10'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl10[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                resnet18(imgin)
                resnet18(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


In [104]:
with open(f'similarities/exhaustive/resnet/unfilled10.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

## VGG

In [112]:
vgg16 = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1).to(device)

In [113]:
vgg16

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [114]:
vgg16.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [115]:
vgg16.classifier[6].register_forward_hook(get_last_layer_activation)

In [116]:
pkl12path = 'mappings/exhaustive_mapping_12.pkl'
pkl12 = utils.load_dict(pkl12path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\12'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if not 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl12[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                vgg16(imgin)
                vgg16(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['reversal']-------------------
-------------Running ['mirror']-------------------
-------------Running ['randomphasic']-------------------
-------------Running ['randomdelete']-------------------
-------------Running ['reversal', 'mirror']-------------------
-------------Running ['randomdelete', 'mirror']-------------------
-------------Running ['reversal', 'randomphasic']-------------------
-------------Running ['randomdelete', 'reversal', 'randomphasic']-------------------
-------------Running ['randomdelete', 'reversal', 'mirror']-------------------


In [119]:
with open(f'similarities/exhaustive/vgg/filled.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [ ]:
pkl11path = 'mappings/exhaustive_mapping_11.pkl'
pkl11 = utils.load_dict(pkl11path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\11'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl11[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                vgg16(imgin)
                vgg16(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

-------------Running ['randominsertion', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'randomphasic']-------------------
-------------Running ['randominsertion', 'reversal', 'mirror']-------------------


In [ ]:
with open(f'similarities/exhaustive/vgg/unfilled11.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

In [ ]:
pkl10path = 'mappings/exhaustive_mapping_10.pkl'
pkl10 = utils.load_dict(pkl10path)
similaritiesdict = defaultdict(list)
last_activations = []
folpath = 'exhaustive_generated_set\\10'
for transformation in utils.HAHN_TRANSFORMATION_COMBOS:
    if 'randominsertion' in transformation:
        print(f'-------------Running {transformation}-------------------')
        for filepath in os.listdir(folpath):
            imgpath = os.path.join(folpath, filepath)
            imgin = Image.open(imgpath)
            imgin = transform(imgin).unsqueeze(0).to(device)

            imglist = pkl10[filepath.split('.')[0]]
            # print(imglist)
            for t in transformation:
                func = getattr(utils, t)
                imglist = func(imglist)
            # print(imglist)
            imgout = utils.array2img(imglist, num_circles=len(imglist), radius=0.75)
            imgout = transform(imgout).unsqueeze(0).to(device)

            with torch.no_grad():
                vgg16(imgin)
                vgg16(imgout)
            assert len(last_activations) == 2
            tensin = last_activations[0].flatten().unsqueeze(0)
            tensout = last_activations[1].flatten().unsqueeze(0)
            sim = torch.nn.functional.cosine_similarity(tensin, tensout, dim=1)
            similaritiesdict[str(transformation)].append(sim)
            last_activations = []

In [ ]:
with open(f'similarities/exhaustive/vgg/unfilled10.pkl', 'wb') as f:
    pickle.dump(similaritiesdict, f)

## DenseNet

In [ ]:
densenet121 = models.densenet121(pretrained=True)

In [ ]:
densenet121

In [ ]:
densenet121.classifier

In [ ]:
densenet121.classifier.register_forward_hook(get_last_layer_activation)

## Inception

In [ ]:
inception_v3 = models.inception_v3(pretrained=True)

In [ ]:
inception_v3

In [ ]:
inception_v3.fc

In [ ]:
inception_v3.fc.register_forward_hook(get_last_layer_activation)

## AlexNet

In [ ]:
alexnet = models.alexnet(pretrained=True)

In [ ]:
alexnet

In [ ]:
alexnet.classifier

In [ ]:
alexnet.classifier[6].register_forward_hook(get_last_layer_activation)